### sobre escreve os dados para corrifir erro 
[PARQUET_TYPE_ILLEGAL] Illegal Parquet type: INT64 (TIMESTAMP(NANOS,false)).
SQLSTATE: 42846



In [0]:
import pyarrow.parquet as pq
import pyarrow as pa

# Read the Parquet file using pyarrow
table = pq.read_table("/Volumes/raw/upcell/cdc/clientes/version=803/")

# Convert all TIMESTAMP(NANOS) columns to string
schema = table.schema
fields = [
    pa.field(
        f.name, 
        pa.string() if pa.types.is_timestamp(f.type) and getattr(f.type, "unit", None) == "ns" else f.type
    ) 
    for f in schema
]
new_schema = pa.schema(fields)
table = table.cast(new_schema)

# Convert to Pandas DataFrame
pdf = table.to_pandas()

# Create Spark DataFrame
df_cdc = spark.createDataFrame(pdf)

df_cdc.write.mode("overwrite").parquet("/Volumes/raw/upcell/cdc/clientes/version=803/")

In [0]:
# Corrigir leitura do CSV
df_corrigido = (
    spark.read.format("csv")
    .option("header", True)
    .option("sep", ";")
    .load("/Volumes/raw/upcell/full-load/clientes/clientes.csv")
)

# Sobrescrever o Parquet com os dados estruturados corretamente
df_corrigido.write.mode("overwrite").format("parquet").save("/Volumes/raw/upcell/full-load/clientes/clientes.parquet")